<a href="https://colab.research.google.com/github/nurfnick/Operations_Research/blob/main/ChapterRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression

## Introduction
Regression is a huge topic but we'll try to cover the basics here with the idea that you will use regression often in business settings and that all the details will not be needed.

We will mostly focus on the Method of Least Squares here.  The idea is to make a prediction and to minimize the error in that prediction by minimizing the sum of the squares of that error:

$$
SSE = \sum_{i=1}^n\left(y_i-\hat {y_i}\right)^2
$$



## Derivation

To derive the linear regression formula, we assume that $\hat y$ is linear,
$$
\hat y = \beta_0 +\beta_1 x
$$

Then we plug this into the sum of squares error and take the partial derivatives with respect to $\beta_0$ and $\beta_1$.  To minimize we then set those derivatives to zero and solve the system.

$$
\nabla SSE(\beta_0,\beta_1) =
\left(
\sum -2(y_i-\beta_0 - \beta_1 x_i)
,
\sum -2(y_i-\beta_0 - \beta_1 x_i)x_i
\right)
$$

Setting each of these to zero and simplifying the summations, we arrive at
$$
\sum y_i = n\beta_0 +\beta_1\sum x_i
$$
$$
\sum x_iy_i = \beta_0\sum x_i +\beta_1\sum x^2_i
$$

Eliminating and solving, we see that
$$
\beta_1 = \frac{n\sum x_iy_i - \sum x_i \sum y_i}{n\sum x_i^2 - (\sum x_i)^2}
$$

and
$$
\beta_0 = \frac{\sum y_i - \beta_1\sum x_i}n
$$